In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
def f():
    print("-------------------------")

In [15]:
df = pd.read_csv("data.csv")
f()
print(f"{df.shape[0]} rows\n{df.shape[1]} columns")

-------------------------
2803 rows
1383 columns


In [16]:
f()
print(f"Columns types distribution:\n\n{df.dtypes.value_counts()}")

-------------------------
Columns types distribution:

float64    1378
object        3
int64         2
dtype: int64


In [69]:
f()
a = df.isna().sum().where(lambda x:x>0).dropna()
print(f"There are {len(a)} (out of {df.shape[1]}, [{round(len(a)/df.shape[1]*100)}%]) columns that contains 1 or more NA")

-------------------------
There are 33 (out of 1383, [2%]) columns that contains 1 or more NA


In [47]:
f()
print(f"NA columns data type Distribution:\n\n{df[a.index].dtypes.value_counts()}")

-------------------------
NA columns data type Distribution:

float64    33
dtype: int64


In [70]:
f()
a = a.sort_values()/len(df)*100
# print(f"Missing values Ratio:(0=No missing value, 100=All values are missing)\n\n{a}")
a == 1

-------------------------


min_call_Duration_sqrt                                         0.035676
min_call_Duration_log                                          0.035676
outgoing_call_logs_weekdays_max_standriezed                  100.000000
outgoind_call_log_COUNT_mean_standriezed                     100.000000
outgoind_call_log_COUNT_median_standriezed                   100.000000
outgoing_call_logs_weekends_mean_standriezed                 100.000000
count_of_days_no_outgoing_call_standriezed                   100.000000
outgoind_call_log_COUNT_min_standriezed                      100.000000
picture_count_Downloads_standriezed                          100.000000
outgoind_call_log_COUNT_max_standriezed                      100.000000
outgoing_call_logs_weekends_max_standriezed                  100.000000
outgoing_call_logs_weekends_std_standriezed                  100.000000
outgoing_call_logs_weekends_min_standriezed                  100.000000
outgoind_call_log_COUNT_std_standriezed                      100

In [63]:
f()
ans = input("Are you need to remove Columns That are all missing?[y|n]")
if ans == "y":
    print(f"data columns BEFORE deleting all NaNs columns: {df.shape[1]}")
    df.drop(columns=a[a==100].index, inplace=True)
    print(f"data columns AFTER deleting all NaNs columns: {df.shape[1]}")

-------------------------


Are you need to remove Columns That are all missing?[y|n] an


Index(['outgoing_call_logs_weekdays_max_standriezed',
       'outgoind_call_log_COUNT_mean_standriezed',
       'outgoind_call_log_COUNT_median_standriezed',
       'outgoing_call_logs_weekends_mean_standriezed',
       'count_of_days_no_outgoing_call_standriezed',
       'outgoind_call_log_COUNT_min_standriezed',
       'picture_count_Downloads_standriezed',
       'outgoind_call_log_COUNT_max_standriezed',
       'outgoing_call_logs_weekends_max_standriezed',
       'outgoing_call_logs_weekends_std_standriezed',
       'outgoing_call_logs_weekends_min_standriezed',
       'outgoind_call_log_COUNT_std_standriezed',
       'outgoing_call_logs_weekdays_mean_standriezed',
       'days_since_last_outgoing_call_standriezed',
       'calendar_events_count_standriezed',
       'outgoing_call_logs_weekdays_std_standriezed', 'video_standriezed',
       'audio_standriezed', 'darkweb_count_ratio_standriezed',
       'dangerous_count_ratio_standriezed', 'darkweb_count_standriezed',
       'danger